In [ ]:
!pip install transformers

In [2]:
import tensorflow as tf
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")

text = """
A SQUAT grey building of only thirty-four stories. Over the main entrance the
words, CENTRAL LONDON HATCHERY AND CONDITIONING CENTRE,
and, in a shield, the World State’s motto, COMMUNITY, IDENTITY, STABILITY.
The enormous room on the ground floor faced towards the north. Cold for
all the summer beyond the panes, for all the tropical heat of the room itself,
a harsh thin light glared through the windows, hungrily seeking some draped
lay figure, some pallid shape of academic goose-flesh, but finding only the glass
and nickel and bleakly shining porcelain of a laboratory. Wintriness responded
to wintriness. The overalls of the workers were white, their hands gloved with
a pale corpse-coloured rubber. The light was frozen, dead, a ghost. Only from
the yellow barrels of the microscopes did it borrow a certain rich and living
substance, lying along the polished tubes like butter, streak after luscious streak
in long recession down the work tables.
“And this,” said the Director opening the door, “is the Fertilizing Room.”
Bent over their instruments, three hundred Fertilizers were plunged, as the Director of Hatcheries and Conditioning entered the room, in the scarcely breathing silence, the absent-minded, soliloquizing hum or whistle, of absorbed
concentration. A troop of newly arrived students, very young, pink and callow,
followed nervously, rather abjectly, at the Director’s heels. Each of them carried
a notebook, in which, whenever the great man spoke, he desperately scribbled.
Straight from the horse’s mouth. It was a rare privilege. The D. H. C. for Central
London always made a point of personally conducting his new students round
the various departments.
“Just to give you a general idea,” he would explain to them. For of course some
sort of general idea they must have, if they were to do their work intelligentlythough as little of one, if they were to be good and happy members of society, as
possible. For particulars, as every one knows, make for virture and happiness;
generalities are intellectually necessary evils. Not philosophers but fretsawyers
""" * 100

tokenized_text = tokenizer.encode(text)

examples = []
block_size = 512
for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
    examples.append(tokenized_text[i:i + block_size])

inputs, labels = [], []
for ex in examples:
    inputs.append(ex[:-1])
    labels.append(ex[1:])

dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))



BATCH_SIZE = 16
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (51200 > 1024). Running this sequence through the model will result in indexing errors


In [2]:
#@title works good without mixed_precision

from transformers import GPT2Tokenizer, TFGPT2LMHeadModel, GPT2Config
from tensorflow.keras.mixed_precision import experimental, global_policy

current_policy = global_policy()
print(f"current_policy: {current_policy}")

config = GPT2Config(
            vocab_size=tokenizer.vocab_size,
            n_positions=512,
            n_ctx=512,
            n_embd=512,
            n_layer=4,
            n_head=4,
            pad_token_id=tokenizer.pad_token_id,
            use_cache=False,
        )
model = TFGPT2LMHeadModel(config)

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])
model.fit(dataset, epochs=20)

current_policy: <Policy "float32">
Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, d

In [3]:
#@title current mixed_precision with ad bad result

policy = experimental.Policy('mixed_float16')
experimental.set_policy(policy)
current_policy = global_policy()
print(f"current_policy: {current_policy}")


config = GPT2Config(
            vocab_size=tokenizer.vocab_size,
            n_positions=512,
            n_ctx=512,
            n_embd=512,
            n_layer=4,
            n_head=4,
            pad_token_id=tokenizer.pad_token_id,
            use_cache=False,
        )
model = TFGPT2LMHeadModel(config)

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])
model.fit(dataset, epochs=20)

Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  Tesla P100-PCIE-16GB, compute capability 6.0
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale
current_policy: <PolicyV1 "mixed_float16", loss_scale=DynamicLossScale(current_loss_scale=32768.0, num_good_steps=0, initial_loss_scale=32768.0, increment_period=2000, multiplier=2.0)>
  opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt)

Epoch 1/20
6/6 [==============================] - 8s 500ms/step - loss: 10.6007 - accuracy: 0.0232
Epoch 2/20
6/6 [==============================] - 3s 500ms

In [ ]:
!pip install https://github.com/mymusise/transformers/archive/fix_mixed_precision4gpt2.zip

In [3]:
#@title mixed_precision after train

from transformers import GPT2Tokenizer, TFGPT2LMHeadModel, GPT2Config
from tensorflow.keras.mixed_precision import experimental, global_policy


policy = experimental.Policy('mixed_float16')
experimental.set_policy(policy)
current_policy = global_policy()
print(f"current_policy: {current_policy}")


config = GPT2Config(
            vocab_size=tokenizer.vocab_size,
            n_positions=512,
            n_ctx=512,
            n_embd=512,
            n_layer=4,
            n_head=4,
            pad_token_id=tokenizer.pad_token_id,
            use_cache=False,
        )
model = TFGPT2LMHeadModel(config)

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])
model.fit(dataset, epochs=20)

Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  Tesla K80, compute capability 3.7
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale
current_policy: <PolicyV1 "mixed_float16", loss_scale=DynamicLossScale(current_loss_scale=32768.0, num_good_steps=0, initial_loss_scale=32768.0, increment_period=2000, multiplier=2.0)>
  opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt)

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full outpu